In [ ]:
from __future__ import print_function

from depmapomics.config import *

from depmapomics import tracker, loading, fusions, expressions
from depmapomics import terra as myterra
from genepy import terra
from genepy.utils import helper as h
import subprocess

import dalmatian as dm

from bokeh.plotting import output_notebook

%load_ext autoreload
%autoreload 2

output_notebook()

In [ ]:
ws = dm.WorkspaceManager('broad-firecloud-ccle/CCLE_PDX_RNAseq').disable_hound()
samples = ws.get_samples()

In [ ]:
terra.changeToBucket(
        samples,
        'gs://coldcclebams/rna/PDX/',
        name_col="index",
        values=['RNAseq_bam'],
        filetypes=['bam'],
        catchdup=True,
        dryrun=False,
    )

In [ ]:
# broad-firecloud-ccle/DepMap_WES_PONS_hg38

In [ ]:
normal_ws = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_WES_PONS_hg38').disable_hound()
normal_samples = normal_ws.get_samples()

In [ ]:
normal_samples

In [ ]:
from google.cloud import storage
import pandas as pd
storage_client = storage.Client()

def check_broken_path(row):
    if pd.isna(row['hg38_analysis_ready_bam']):
        return False
    else:
        bucket_name = row['hg38_analysis_ready_bam'].split("/")[2]
        file_name = "/".join(row['hg38_analysis_ready_bam'].split("/")[3:])
        bucket = storage_client.bucket(bucket_name)
        return storage.Blob(bucket=bucket, name=file_name).exists(storage_client)

In [ ]:
validnormals = normal_samples[normal_samples.apply(check_broken_path, axis=1)]
validnormals

In [ ]:
terra.changeToBucket(
        validnormals,
        'gs://depmap-wes-normals/',
        name_col="index",
        values=['hg38_analysis_ready_bam', 'hg38_analysis_ready_bam_index'],
        filetypes=['bam', 'bai'],
        catchdup=True,
        dryrun=False,
    )

In [ ]:
samples = terra.changeToBucket(
        validnormals,
        'gs://depmap-wes-normals/',
        name_col="index",
        values=['hg38_analysis_ready_bam', 'hg38_analysis_ready_bam_index'],
        filetypes=['bam', 'bai'],
        catchdup=True,
        dryrun=True,
    )

In [ ]:
samples

In [ ]:
for i in normal_samples.index:
    if i in samples.index and normal_samples.loc[i, 'hg38_analysis_ready_bam_index'] != samples.loc[i, 'hg38_analysis_ready_bam_index']:
        normal_samples.loc[i, 'hg38_analysis_ready_bam'] = samples.loc[i, 'hg38_analysis_ready_bam']
        normal_samples.loc[i, 'hg38_analysis_ready_bam_index'] = samples.loc[i, 'hg38_analysis_ready_bam_index']
normal_samples

In [ ]:
normal_samples.loc['AGILENT_10541']

In [ ]:
samples.loc['AGILENT_10541', 'hg38_analysis_ready_bam_index']

In [ ]:
normal_ws.update_sample_attributes(normal_samples)

In [ ]:
torm = await terra.deleteHeavyFiles("broad-firecloud-ccle/DepMap_WES_PONS_hg38")
h.parrun(['gsutil rm '+i for i in storm], cores=8)

In [ ]:
rna_wm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_hg38_RNAseq').disable_hound()
rna_samples = rna_wm.get_samples()

In [ ]:
# too slow!! because changeToBucket uses cp instead of mv
rna_samples = terra.changeToBucket(
        rna_samples,
        'gs://cclebams/rna_transcriptome/',
        name_col="index",
        values=['star_transcriptome_bam'],
        filetypes=['bam'],
        catchdup=True,
        dryrun=True,
    )

In [ ]:
from genepy.google import gcp

gcp.mvFiles(rna_samples.star_transcriptome_bam.tolist(), 'gs://cclebams/rna_transcriptome/')

In [ ]:
for i in rna_samples.index:
    if not (rna_samples.loc[i, 'star_transcriptome_bam'].startswith('gs://cclebams/')):
        rna_samples.loc[i, 'star_transcriptome_bam'] = 'gs://cclebams/rna_transcriptome/' + rna_samples.loc[i, 'star_transcriptome_bam'].split('/')[-1]

In [ ]:
rna_samples.star_transcriptome_bam

In [ ]:
rna_wm.update_sample_attributes(rna_samples)